# **Lab Assignment 4**
**Creator**: Lauren Enriquez  
**Date**: October 2, 2019  

The purpose of this lab is to create a database with genes from glycolysis, the citrate cycle, and the pentose phosphate pathway.<br>
Four enzymes were selected from Drosophila, E. coli, and humans. <br>

In [ ]:
# Wrappers for supported file formats
import string
import sqlite3
from Bio import Entrez
from Bio import SeqIO
from Bio import SeqFeature
import pandas as pd
import sys
import urllib.request as ur
from sqlite3 import Error

### Establish connection to database

In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:            
        print(e)
    finally:
        conn.close()

if __name__ == '__main__':
    create_connection("/data/home/5539626/BioE131-231/Lab_4/Lab_4_database.db")

### **Create Genes table, Pathways table, and Enzyme table**

In [ ]:
conn = sqlite3.connect('Lab_4_database.db')
c = conn.cursor()

In [ ]:
c.execute("DROP TABLE genes;")
c.execute("DROP TABLE pathway;")
c.execute("DROP TABLE enzyme;")
c.execute("DROP TABLE enzyme_pathway;")

In [ ]:
# Creates Gene Table
c.execute("""CREATE TABLE genes (id REAL,
                                 name TEXT,
                                 description TEXT,
                                 organism TEXT,
                                 nucleotide_seq TEXT);""")

# Creates Pathway Table
c.execute("""CREATE TABLE pathway (id INT,
                                   name TEXT,
                                   description TEXT);""")

# Creates Enzyme Table
c.execute("""CREATE TABLE enzyme (id INT,
                                  name TEXT,
                                  function TEXT,
                                  ECnum INT,
                                  cycle TEXT);""")

# Save (commit) the changes
conn.commit()

### **Data Collection**
All data is from Entrez nucleotide and gene databases: <br> https://www.ncbi.nlm.nih.gov/search/?utm_expid=.fBQRRb8XTVS1Ew6CREtJgg.0&utm_referrer=https%3A%2F%2Fwww.ncbi.nlm.nih.gov%2FWeb%2FSearch%2Fentrezfs.html

Kegg was used to select the enzymes used for ths lab:<br>
https://www.genome.jp/kegg-bin/show_pathway?map00010

In [ ]:
Entrez.email = 'lauren_enriquez@berkeley.edu'
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
handle.close()

In [ ]:
kegg_hsa = "http://rest.kegg.jp/link/hsa/hsa00010"
kegg_eco = "http://rest.kegg.jp/link/eco/eco00010"
kegg_dme = "http://rest.kegg.jp/link/dme/dme00010"

filehandler = ur.urlopen (kegg_hsa) 

test = []

for line in filehandler:
    test.append(str(line).strip("b'"))
    line = str(line).replace("t","")

In [ ]:
handle = Entrez.esummary(db = "pubmed",
                         id = "19304878,14630660",
                         retmode="xml") 

records = Entrez.parse(handle)

In [ ]:
handle = Entrez.efetch(db = "nucleotide",
                       id = "AY851612",
                       rettype = "gb",
                       retmode="text")

In [ ]:
handle = Entrez.esearch(db = "nucleotide",
                        term = "homo sapiens[ORGN] BCRA1",
                        sort = "relevance",
                        idtype = "acc")

for i in Entrez.read(handle)["IdList"]:
    handle = Entrez.efetch(db ="nucleotide",
                           id = i,
                           rettype ="fasta",
                           retmode="text")

The **above code** returns data as GenBank and using Bio.SeqIO to read the returned GenBank file.<br><br>
>**print(handle.read( ))**

U11292.1 Homo sapiens Ki nuclear autoantigen mRNA, complete cds
GGGCGGACAGGCACAGAGGGAGGGAGCGAGCGAGCAGTGAGTAAGCCAGCAAGGGCGGTCGGGTCCCGAG
GTCAGCCGAGATTTCTCAGGTCCCTCCGGCCCCCTCCCTGGAGTCCACAGCGCCTCCGGTGTCCAGAGGA
TCGGACACGGCCCGGCCCGGCCATGGCCTCGTTGCTGAAGGTGGATCAGGAAGTGAAGCTCAAGGTTGAT
TCTTTCAGGGAGCGGATCACAAGTAAGGCAGAAGACTTGGTGGCAAATTTTTTCCCAAAGAAGTTATTAG
AACTTGATAGTTTTCTGAAGGAACCAATCTTAAACATCCATGACCTAACTCAGATCCACTCTGACATGAA
TCTCCCAGTCCCTGACCCCATTCTTCTCACCAATAGCCATGATGGACTGGATGGTCCCACTTATAAGAAG
CGAAGGTTGGATGAGTGTGAAGAAGCCTTCCAAGGAACCAAGGTGTTTGTGATGCCCAATGGGATGCTGA
AAAGCAACCAGCAGCTGGTGGACATTATTGAGAAAGTGAAACCTGAGATCCGGCTGTTGATTGAGAAATG
TAACACGCCTTCAGGCAAAGGTCCTCATATATGTTTTGACCTCCAGGTCAAAATGTGGGTACAGCTCCTG
ATTCCCAGGATAGAAGATGGAAACAACTTTGGGGTGTCCATTCAGGAGGAAACAGTTGCAGAGCTAAGAA
CTGTTGAGAGTGAAGCTGCATCTTATCTGGACCAGATTTCTAGATATTATATTACAAGAGCCAAATTGGT
TTCTAAAATAGCTAAATATCCCCATGTGGAGGACTATCGCCGCACCGTGACAGAGATTGATGAGAAAGAA
TATATCAGCCTTCGGCTCATCATATCAGAGCTGAGGAATCAATATGTCACTCTACATGACATGATCCTGA
AAAATATCGAGAAGATCAAACGGCCCCGGAGCAGCAATGCAGAGACTCTGTACTGAGGCCAGGGCCAGGG
CCAGGGGACTCTGTGAGTCTGGCTCAAGACCGACATTGCCTTGGTTTGTTACATGACTATCGTGATGGGG
AAACTGGCTGGAAATAGTAATCACACCTCTCTGTTTTTAGTTAGAGTCTAATGAAACTCTCATCTAGTTC
TGTGATGTGTTTACCTCTTTTTTCAGGCCTCAGGAACTCTTCTATTTCCTTCCCTAATACCCCACACCCA
ACCTGTCGTAATTTCTGGAGAACTCCAGGTTTGTGTGTGCAGGATGTTGGCACAAAAATACCTGTGTTTT
CATTCTCCCCCTCTCTCCCTCCTGTGTCTGGCGCTTTATGTTTTCTTCCGTTTGATAATTAGTTGGTTAA
AAGCTGAGGGAACCGGAAGGAAAGTGCTAGGTGTTTTTTAGGAACTAGGGTGGAGGGGGGACGAACTTCT
CTTCCTCACATGAGGTTACTGTTTCTTTCCTCTGTGGGGCATTGGATCCTCCCACAGTTGCCCTGGTGAT
GACTTAGGACTTCCCATCTGTGACATCCCACTTTGAATCTTGATCGTGACAAGAAATACCTTAGGCCTTC
AGTCAATTCCGAAGCTCCTTCAGTTGTTTTTATAATGGGCGTTTCACATGCACATATGTGTATGCATGTA
TACGCCCATACAGACATGCACACACAGACTCCTACTCCATTAGCTAACATACCCTCCCTCTCCACAACCC
GTGTCACATACCTTTCAGGAGGTGACAGTTGTCTTAGTTGTCATCTACCCAGACAAACGTCCTGGGCCCG
TCCTCCCTCCTGATACTGTAGCCTCTTGGTACCCAGGGTGAGTTGGTGGAGAACAGAGAGATGAGAAGCA
GAGGGCTTGGGGAAAGCCTGTTCCTCTCTGACTCAGCCCTTTTTGGCATTATTGCAAGAGCTTGACTCCT
GGTTGCCTTTTCCCAGCCAGTTTTCAGTTGGGGTGAAGGTTTCTGCAAGTGTGAGGTCCAGATGCTGCTG
CTCATGTTGGGCTTTCCTTTTGGGAACTATTTCTCTTTATTTATAGTGTCGGGCTTCCGGGGAAAGCAAT
CATTGGTGTGTATGTGTATGTGCCATGCACACACGTGCATATATACACATTTGTGTATGTGGAAATGTGC
TGGGCAAGTCAAAACTATAGAAGAGTTGCCTCCTGTCTCTCGAATCTTCCAGAGATATCACTTAATTGTT
AACAGCTTTTGTGTTAATCCCCTTCATCCCCTAGCACTTTTATTCTACCACGGCTGGAGAGTTGANANCT
ACAGTCAGCCTGCCAGTGACTCTTAGTGTCTGTTTCTGACTTATTTTTCCTGTCTCTGTCTTCCAACCCC
CAATAATATTTCCCACCGGGGATGCATCATTTTTACTCCCAATATTCTGTAGAGAGGGAGTCAGGATGCT
GTCTTCCCACGAATAGTACTCAGTAACAAACCAATTGCATTTTAGTTGGGCAGTGCTCCCACCCACCCTG
CAGATCCCTCCAGCTAAAACCCTTCCCCCTTCCCTCCATGTGTTTCTCAGTTTCCCGTTCGTTTGTTGGA
CTGTTCCACTGCCCCTCCTCCTCACCCTATCACCCATGGATCGTAATGTAAAATTCTTTTACCATGTCAA
GAAATTATTAAAAATACAGGTACTTTGACCTCTTTCTAAAGCCGCAGACCCTGGTGCAATGCTCTGGTGG
CTAGGGATGTACTCATGCTCATATGTGTGCACGCTTGGACACCCACCTCCATGGACACCTAGCCACCCTG
TTGTGTGNCCTTATGCCAGTTGAGCTGAATCTTTTCCCCAGTATAGTGGAAAGACTGAGGCTTCTGCCTA
CTGAGCAAGGTTGGGTGCTTCATTTGTGTTCAGTCTGAATTATGGGAAAGTTAGCTCTTCCCAGACCTAA
GCTGCCTTCTCTCCCTACTTTCAG

### **Fill Pathways Table**

In [ ]:
c.execute("""INSERT INTO pathway VALUES('1',
                                         'glycolysis',
                                         'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).');""")

c.execute("""INSERT INTO pathway VALUES('2',
                                        'TCA cycle',
                                        'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.');""")

c.execute("""INSERT INTO pathway VALUES('3',
                                        'pentose phosphate',
                                        'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.');""")
conn.commit()

### **Pick 4 enzymes from glycolysis, TCA cycle, and pentose phosphate pathways**

In [ ]:
glycolysis_enz = ['Drosophila[ORGN] 5.1.3.3',
                  'Drosophila[ORGN] 2.7.1.1',
                  'Drosophila[ORGN] 3.1.3.9',
                  'Drosophila[ORGN] 5.3.1.9',
                  'Escherichia coli[ORGN] 5.1.3.3',
                  'Escherichia coli[ORGN] 5.3.1.1',
                  'Escherichia coli[ORGN] 5.3.1.9',
                  'Escherichia coli[ORGN] 3.1.3.11',
                  'Homo sapiens[ORGN] 4.1.2.13',
                  'Homo sapiens[ORGN] 2.7.1.1',
                  'Homo sapiens[ORGN] 3.1.3.9',
                  'Homo sapiens[ORGN] 5.3.1.9']

TCA_enz = ['Drosophila[ORGN] 4.1.1.32', 
           'Drosophila[ORGN] 2.3.1.12', 
           'Drosophila[ORGN] 1.2.4.1',
           'Drosophila[ORGN] 2.3.3.1',
           'Escherichia coli[ORGN] 4.1.1.49',
           'Escherichia coli[ORGN] 2.3.3.1',
           'Escherichia coli[ORGN] 1.2.7.1',
           'Escherichia coli[ORGN] 4.2.1.3',
           'Homo sapiens[ORGN] 4.1.1.32', 
           'Homo sapiens[ORGN] 2.3.1.12', 
           'Homo sapiens[ORGN] 1.2.4.1',
           'Homo sapiens[ORGN] 2.3.3.1']

pentose_phosphate_enz = ['Drosophila[ORGN] 1.1.1.49',
                         'Drosophila[ORGN] 5.3.1.9',
                         'Drosophila[ORGN] 1.1.5.9',
                         'Drosophila[ORGN] 1.1.1.44',
                         'Escherichia coli[ORGN] 1.1.1.49', 
                         'Escherichia coli[ORGN] 2.2.1.1',
                         'Escherichia coli[ORGN] 3.1.1.31', 
                         'Escherichia coli[ORGN] 5.3.1.6',
                         'Homo sapiens[ORGN] 3.1.1.17', 
                         'Homo sapiens[ORGN] 3.1.1.31', 
                         'Homo sapiens[ORGN] 5.1.3.1', 
                         'Homo sapiens[ORGN] 1.1.1.49']

### **Create function to fill enzyme table**
Immediately after, the function is called to fill in the table

In [ ]:
def enzyme_table(cycle_enzymes, cycle): 
    enzymes = []
    ECnums = []
    count = 0
    for enzyme in cycle_enzymes:
        handle1 = Entrez.esearch(db = 'protein', 
                                 term = enzyme, 
                                 sort = 'relevance', 
                                 idtype = 'acc')
        record = Entrez.read(handle1)['IdList']
        enzymes.append(record[0])
        ECnum = (enzyme.split()[-1])
        ECnums.append(ECnum)

    for ID in enzymes:
        handle2 = Entrez.efetch(db = 'protein', 
                                id = ID, 
                                rettype ='gb', 
                                retmode ='text')
        value = SeqIO.read(handle2, 'gb')
        c.execute(""" INSERT INTO enzyme (id,
                                          name, 
                                          function,
                                          ECnum,
                                          cycle) 
                                          VALUES(?,?,?,?,?)""", 
                                          (value.id, 
                                           value.name, 
                                           value.description, 
                                           ECnums[count],
                                           cycle))
        count += 1

In [ ]:
enzyme_table(glycolysis_enz, "Glycolysis")
conn.commit()

In [ ]:
enzyme_table(TCA_enz, "TCA")
conn.commit()

In [ ]:
enzyme_table(pentose_phosphate_enz, "Pentose Phosphate")
conn.commit()

### **Create function to fill genes table**
Immediately after, the function is called three times to fill in the table

In [ ]:
def gene_table(cycle_enzymes):
    genes = []
    names = []
    descriptions = []
    chromo = []
    idd = []
    organisms = []
    sequences = []
    
    for gene in cycle_enzymes:
        handle = Entrez.esearch(db = 'nuccore',
                                 term = gene, 
                                 sort = 'relevance', 
                                 idtype ='acc')
        record = Entrez.read(handle)['IdList'][:1]
        genes.append(record)
        
    for gene in genes:
        handle1 = Entrez.efetch(db = 'nuccore', 
                                id = gene,
                                sort = 'relevance',
                                rettype = 'gb',
                                retmode = 'txt')
        seq_record = SeqIO.read(handle1, 'genbank')
        seq_id = seq_record.id
        idd.append(seq_id)
        
        sequence = str(seq_record.seq)
        sequences.append(sequence)
        
        name = seq_record.name
        names.append(name)
        
        desc = seq_record.description
        descriptions.append(desc)

        annotations = seq_record.annotations
        source = seq_record.annotations["organism"]
        organisms.append(source)
    
    i = 0
    for ID in genes: 
        c.execute(""" INSERT INTO genes (id,
                                         name,
                                         description,
                                         organism,
                                         nucleotide_seq) 
                                         VALUES (?,?,?,?,?)""", 
                                         (idd[i],
                                          names[i], 
                                          descriptions[i],
                                          organisms[i],
                                          sequences[i]))
        i += 1

In [ ]:
gene_table(glycolysis_enz)
conn.commit()

In [ ]:
gene_table(TCA_enz)
conn.commit()

In [ ]:
gene_table(pentose_phosphate_enz)
conn.commit()

***
For unknown reasons, the portion of gene data for the pentose phosphate pathway kept giving an error. The specific error was for "No records in handle". It **orginally** completed the function, but **recently** it has given this error.

### **Create Association Table (Enzyme / Pathway)**

In [ ]:
#c.execute("DROP TABLE enzyme_pathway;")
c.execute("""CREATE TABLE enzyme_pathway AS SELECT id, ECnum, cycle FROM enzyme""")
conn.commit()

### **Prints Enzyme Table**
To save space, only the first **3 entries** were printed in markdown. <br>
>**for row in c.execute("""SELECT * FROM enzyme"""): <br>
    print(row) <br>
    print('') <br>**
    
('NP_730671.1', 'NP_730671', 'uncharacterized protein Dmel_CG32445 [Drosophila melanogaster]', '5.1.3.3', 'Glycolysis')

('XP_002055038.1', 'XP_002055038', 'uncharacterized protein Dvir_GJ19157 [Drosophila virilis]', '2.7.1.1', 'Glycolysis')

('NP_001097063.1', 'NP_001097063', 'Glucose-6-Phosphatase [Drosophila melanogaster]', '3.1.3.9', 'Glycolysis')


### **Prints Pathway Table**
>**for row in c.execute("""SELECT * FROM pathway"""): <br>
    print(row)<br>
    print('')<br>**
    
(1, 'glycolysis', 'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).')

(2, 'TCA cycle', 'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.')

(3, 'pentose phosphate', 'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.')

### **Prints Association Table**
#### Enzyme and Pathway relationship
There are many relationships between enzyme and pathway. Multiple unique enzymes are in the same pathway. Enzymes with the same EC number come from the same pathway. **An example was dissected from the data printed below**.<br>
>**for row in c.execute("""SELECT * FROM enzyme_pathway"""): <br>
    print(row) <br>
    print('') <br>**

('XP_002048320.1', '1.1.1.49', 'Pentose Phosphate')

('P0AC53.1', '1.1.1.49', 'Pentose Phosphate')

('AAA52501.1', '1.1.1.49', 'Pentose Phosphate')

('XP_002048320.1', '1.1.1.49', 'Pentose Phosphate')

('P0AC53.1', '1.1.1.49', 'Pentose Phosphate')

('AAA52501.1', '1.1.1.49', 'Pentose Phosphate')

### **Tests the view of all the tables**

print (pd.read_sql_query("SELECT * FROM genes;",conn)) <br>
print (pd.read_sql_query("SELECT * FROM enzyme;", conn))<br>
print (pd.read_sql_query("SELECT * FROM pathway;", conn))<br>
print (pd.read_sql_query("SELECT * FROM enzyme_pathway;", conn))<br>

In [ ]:
conn.close()